# I’m Something of a Painter Myself - Visualization and Augmentation

This notebook demonstrates different approaches to exploratory data analysis for the image data.   
In this solution, you will also see how you can visualize images using matplotlib and what graphs you can build for these images

![](https://storage.googleapis.com/kaggle-competitions/kaggle/21755/logos/header.png)

<a id="top"></a>

<div class="list-group" id="list-tab" role="tablist">
<h3 class="list-group-item list-group-item-action active" data-toggle="list" style='background:purple; border:0' role="tab" aria-controls="home"><center>Quick Navigation</center></h3>

* [1. Data Preparations](#1)
* [2. Show Batches - Monet Paintings](#2)
* [3. Show Batches - Photos](#3)
* [4. Color Channel Histograms](#4)
* [5. Individual Channels Visualization](#5)
* [6. Grayscale Visualization](#6)
    

* [Augmentation - Blur](#100)
* [Augmentation - CLAHE](#101)
* [Augmentation - CenterCrop](#102)
* [Augmentation - ChannelDropout](#103)
* [Augmentation - ChannelShuffle](#104)
* [Augmentation - Crop](#105)
* [Augmentation - RandomCrop](#106)
* [Augmentation - CutOut](#107)
* [Augmentation - Downscale](#108)
* [Augmentation - Equalize](#109)
* [Augmentation - HorizontalFlip](#110)
* [Augmentation - VerticalFlip](#111)
* [Augmentation - Flip](#112)
* [Augmentation - GaussNoise](#113)
* [Augmentation - GridDistortion](#114)    
* [Augmentation - HueSaturationValue](#115) 
* [Augmentation - ISONoise](#116)
* [Augmentation - ImageCompression](#117)
* [Augmentation - InvertImg](#118)
* [Augmentation - JpegCompression](#119)
* [Augmentation - MotionBlur](#120)
* [Augmentation - MultiplicativeNoise](#121)

<a id="1"></a>
<h2 style='background:purple; border:0; color:white'><center>Data Preparations<center><h2>

In [ ]:
import os
import math
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import albumentations as A

In [ ]:
def set_seed(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    
    
SEED = 42
set_seed(SEED)

In [ ]:
BASE_PATH = "../input/gan-getting-started/"
MONET_PATH = os.path.join(BASE_PATH, "monet_jpg")
PHOTO_PATH = os.path.join(BASE_PATH, "photo_jpg")

In [ ]:
def print_folder_statistics(path):
    d_image_sizes = {}
    for image_name in os.listdir(path):
        image = cv2.imread(os.path.join(path, image_name))
        d_image_sizes[image.shape] = d_image_sizes.get(image.shape, 0) + 1
        
    for size, count in d_image_sizes.items():
        print(f"shape: {size}\tcount: {count}")


print(f"Monet images:")
print_folder_statistics(MONET_PATH)
print("-" * 10)
print(f"Photo images:")
print_folder_statistics(PHOTO_PATH)
print("-" * 10)

In [ ]:
def batch_visualization(path, n_images, is_random=True, figsize=(16, 16)):
    plt.figure(figsize=figsize)
    
    w = int(n_images ** .5)
    h = math.ceil(n_images / w)
    
    all_names = os.listdir(path)
    
    image_names = all_names[:n_images]
    if is_random:
        image_names = random.sample(all_names, n_images)
    
    for ind, image_name in enumerate(image_names):
        img = cv2.imread(os.path.join(path, image_name))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
        plt.subplot(h, w, ind + 1)
        plt.imshow(img)
        plt.axis("off")
    
    plt.show()

<a id="2"></a>
<h2 style='background:purple; border:0; color:white'><center>Show Batches - Monet Paintings<center><h2>

In [ ]:
batch_visualization(MONET_PATH, 1, is_random=True, figsize=(5, 5))

In [ ]:
batch_visualization(MONET_PATH, 4, is_random=True, figsize=(10, 10))

In [ ]:
batch_visualization(MONET_PATH, 9, is_random=True)

In [ ]:
batch_visualization(MONET_PATH, 16, is_random=True)

In [ ]:
batch_visualization(MONET_PATH, 300, is_random=False)

<a id="3"></a>
<h2 style='background:purple; border:0; color:white'><center>Show Batches - Photos<center><h2>

In [ ]:
batch_visualization(PHOTO_PATH, 16, is_random=True)

<a id="4"></a>
<h2 style='background:purple; border:0; color:white'><center>Color Channel Histograms<center><h2>


In [ ]:
def color_hist_visualization(image_path, figsize=(16, 4)):
    plt.figure(figsize=figsize)
    
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
    plt.subplot(1, 4, 1)
    plt.imshow(img)
    plt.axis("off")
    
    colors = ["red", "green", "blue"]
    for i in range(len(colors)):
        plt.subplot(1, 4, i + 2)
        plt.hist(
            img[:, :, i].reshape(-1),
            bins=25,
            alpha=0.5,
            color=colors[i],
            density=True
        )
        plt.xlim(0, 255)
        plt.xticks([])
        plt.yticks([])
    
    
    plt.show()

In [ ]:
img_path = '../input/gan-getting-started/monet_jpg/000c1e3bff.jpg'
color_hist_visualization(img_path)

img_path = '../input/gan-getting-started/monet_jpg/05144e306f.jpg'
color_hist_visualization(img_path)

img_path = '../input/gan-getting-started/monet_jpg/16dabe418c.jpg'
color_hist_visualization(img_path)

<a id="5"></a>
<h2 style='background:purple; border:0; color:white'><center>Individual Channels Visualization<center><h2>


In [ ]:
def channels_visualization(image_path, figsize=(16, 4)):
    plt.figure(figsize=figsize)
    
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
    plt.subplot(1, 4, 1)
    plt.imshow(np.mean(img, axis=2), cmap="gray")
    plt.axis('off')
    
    for i in range(3):
        plt.subplot(1, 4, i + 2)
        tmp_img = np.full_like(img, 0)
        tmp_img[:, :, i] = img[:, :, i]
        plt.imshow(tmp_img)
        plt.xlim(0, 255)
        plt.xticks([])
        plt.yticks([])
    
    
    plt.show()

In [ ]:
img_path = '../input/gan-getting-started/monet_jpg/51db3fc011.jpg'
color_hist_visualization(img_path)
channels_visualization(img_path)

img_path = '../input/gan-getting-started/monet_jpg/1814cc6632.jpg'
color_hist_visualization(img_path)
channels_visualization(img_path)

img_path = '../input/gan-getting-started/monet_jpg/4995c04b1a.jpg'
color_hist_visualization(img_path)
channels_visualization(img_path)

<a id="6"></a>
<h2 style='background:purple; border:0; color:white'><center>Grayscale Visualization<center><h2>


In [ ]:
def grayscale_visualization(image_path, figsize=(8, 4)):
    plt.figure(figsize=figsize)
    
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
    plt.subplot(1, 2, 1)
    plt.imshow(img)
    plt.axis('off')
    
    plt.subplot(1, 2, 2)
    tmp_img = np.full_like(img, 0)
    for i in range(3):
        tmp_img[:, :, i] = img.mean(axis=-1)
    plt.imshow(tmp_img)
    plt.axis('off')
    
    
    plt.show()

In [ ]:
img_path = '../input/gan-getting-started/monet_jpg/5c79cfe0b3.jpg'
grayscale_visualization(img_path)

img_path = '../input/gan-getting-started/monet_jpg/990ed28f62.jpg'
grayscale_visualization(img_path)

img_path = '../input/gan-getting-started/monet_jpg/fd63a333f1.jpg'
grayscale_visualization(img_path)

<a id="100"></a>
<h2 style='background:purple; border:0; color:white'><center>Augmentation - Blur<center><h2>

In [ ]:
def plot_simple_augmentation(image_path, transform):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
    
    plt.figure(figsize=(10, 5))
    
    plt.subplot(1, 2, 1)
    plt.imshow(img)
    plt.axis("off")
    
    plt.subplot(1, 2, 2)
    x = transform(image=img)["image"]
    plt.imshow(x)
    plt.axis("off")

    plt.show()
    
def plot_multiple_augmentation(image_path, transform):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
    
    plt.figure(figsize=(10, 10))
    
    plt.subplot(2, 2, 1)
    plt.imshow(img)
    plt.axis("off")
    
    plt.subplot(2, 2, 2)
    x = transform(image=img)["image"]
    plt.imshow(x)
    plt.axis("off")
    
    plt.subplot(2, 2, 3)
    x = transform(image=img)["image"]
    plt.imshow(x)
    plt.axis("off")
    
    plt.subplot(2, 2, 4)
    x = transform(image=img)["image"]
    plt.imshow(x)
    plt.axis("off")

    plt.show()

In [ ]:
transform = A.Compose(
    [
        A.Blur(p=1.0, blur_limit=(5, 5)),
    ]
)

plot_simple_augmentation("../input/gan-getting-started/monet_jpg/990ed28f62.jpg", transform)

<a id="101"></a>
<h2 style='background:purple; border:0; color:white'><center>Augmentation - CLAHE<center><h2>

In [ ]:
transform = A.CLAHE(p=1.0, clip_limit=(10, 10), tile_grid_size=(3, 3))

plot_simple_augmentation("../input/gan-getting-started/monet_jpg/990ed28f62.jpg", transform)

<a id="102"></a>
<h2 style='background:purple; border:0; color:white'><center>Augmentation - CenterCrop<center><h2>

In [ ]:
transform = A.CenterCrop(p=1.0, height=100, width=150)

plot_simple_augmentation("../input/gan-getting-started/monet_jpg/990ed28f62.jpg", transform)

<a id="103"></a>
<h2 style='background:purple; border:0; color:white'><center>Augmentation - ChannelDropout<center><h2>

In [ ]:
transform = A.ChannelDropout(p=1.0, channel_drop_range=(1, 2), fill_value=0)

plot_multiple_augmentation("../input/gan-getting-started/monet_jpg/990ed28f62.jpg", transform)

<a id="104"></a>
<h2 style='background:purple; border:0; color:white'><center>Augmentation - ChannelShuffle<center><h2>

In [ ]:
transform = A.ChannelShuffle(p=1.0)

plot_multiple_augmentation("../input/gan-getting-started/monet_jpg/990ed28f62.jpg", transform)

<a id="105"></a>
<h2 style='background:purple; border:0; color:white'><center>Augmentation - Crop<center><h2>

In [ ]:
transform = A.Crop(p=1.0, x_min=0, y_min=0, x_max=150, y_max=150)

plot_simple_augmentation("../input/gan-getting-started/monet_jpg/990ed28f62.jpg", transform)

<a id="106"></a>
<h2 style='background:purple; border:0; color:white'><center>Augmentation - RandomCrop<center><h2>

In [ ]:
transform = A.RandomCrop(p=1.0, height=100, width=100)

plot_multiple_augmentation("../input/gan-getting-started/monet_jpg/990ed28f62.jpg", transform)

<a id="107"></a>
<h2 style='background:purple; border:0; color:white'><center>Augmentation - CutOut<center><h2>

In [ ]:
transform = A.Cutout(p=1.0, num_holes=8, max_h_size=15, max_w_size=15)

plot_multiple_augmentation("../input/gan-getting-started/monet_jpg/990ed28f62.jpg", transform)

<a id="108"></a>
<h2 style='background:purple; border:0; color:white'><center>Augmentation - Downscale<center><h2>

In [ ]:
transform = A.Downscale(
    p=1.0, scale_min=0.01, scale_max=0.20, interpolation=0,
)

plot_multiple_augmentation(
    "../input/gan-getting-started/monet_jpg/990ed28f62.jpg",
    transform,
)

<a id="109"></a>
<h2 style='background:purple; border:0; color:white'><center>Augmentation - Equalize<center><h2>

In [ ]:
transform = A.Equalize(
    p=1.0, mode='cv', by_channels=True,
)

plot_simple_augmentation(
    "../input/gan-getting-started/monet_jpg/990ed28f62.jpg",
    transform,
)

<a id="110"></a>
<h2 style='background:purple; border:0; color:white'><center>Augmentation - HorizontalFlip<center><h2>

In [ ]:
transform = A.HorizontalFlip(
    p=1.0,
)

plot_simple_augmentation(
    "../input/gan-getting-started/monet_jpg/990ed28f62.jpg",
    transform,
)

<a id="111"></a>
<h2 style='background:purple; border:0; color:white'><center>Augmentation - VerticalFlip<center><h2>

In [ ]:
transform = A.VerticalFlip(
    p=1.0,
)

plot_simple_augmentation(
    "../input/gan-getting-started/monet_jpg/990ed28f62.jpg",
    transform,
)

<a id="112"></a>
<h2 style='background:purple; border:0; color:white'><center>Augmentation - Flip<center><h2>

In [ ]:
transform = A.Flip(
    p=1.0,
)

plot_multiple_augmentation(
    "../input/gan-getting-started/monet_jpg/990ed28f62.jpg",
    transform,
)

<a id="113"></a>
<h2 style='background:purple; border:0; color:white'><center>Augmentation - GaussNoise<center><h2>

In [ ]:
transform = A.GaussNoise(
    p=1.0, var_limit=(500.0, 500.0),
)

plot_simple_augmentation(
    "../input/gan-getting-started/monet_jpg/990ed28f62.jpg",
    transform,
)

<a id="114"></a>
<h2 style='background:purple; border:0; color:white'><center>Augmentation - GridDistortion<center><h2>

In [ ]:
transform = A.GridDistortion(
    p=1.0, num_steps=15, distort_limit=(-2., 2.), 
    interpolation=0, border_mode=0, value=(0, 0, 0), mask_value=None,
)

plot_multiple_augmentation(
    "../input/gan-getting-started/monet_jpg/990ed28f62.jpg",
    transform,
)

<a id="115"></a>
<h2 style='background:purple; border:0; color:white'><center>Augmentation - HueSaturationValue<center><h2>

In [ ]:
transform = A.HueSaturationValue(
    p=1.0, 
    hue_shift_limit=(-100, 100), 
    sat_shift_limit=(-100, 100), 
    val_shift_limit=(-100, 100),
)

plot_multiple_augmentation(
    "../input/gan-getting-started/monet_jpg/990ed28f62.jpg",
    transform,
)

<a id="116"></a>
<h2 style='background:purple; border:0; color:white'><center>Augmentation - ISONoise<center><h2>

In [ ]:
transform = A.ISONoise(
    p=1.0, intensity=(0.0, 2.0), color_shift=(0.0, 1.0)
)

plot_multiple_augmentation(
    "../input/gan-getting-started/monet_jpg/990ed28f62.jpg",
    transform,
)

<a id="117"></a>
<h2 style='background:purple; border:0; color:white'><center>Augmentation - ImageCompression<center><h2>


In [ ]:
transform = A.ImageCompression(
    p=1.0, quality_lower=0, quality_upper=10, compression_type=0,
)

plot_multiple_augmentation(
    "../input/gan-getting-started/monet_jpg/990ed28f62.jpg",
    transform,
)

<a id="118"></a>
<h2 style='background:purple; border:0; color:white'><center>Augmentation - InvertImg<center><h2>


In [ ]:
transform = A.InvertImg(
    p=1.0,
)

plot_simple_augmentation(
    "../input/gan-getting-started/monet_jpg/990ed28f62.jpg",
    transform,
)

<a id="119"></a>
<h2 style='background:purple; border:0; color:white'><center>Augmentation - JpegCompression<center><h2>



In [ ]:
transform = A.JpegCompression(
    p=1.0, quality_lower=0, quality_upper=10,
)

plot_multiple_augmentation(
    "../input/gan-getting-started/monet_jpg/990ed28f62.jpg",
    transform,
)

<a id="120"></a>
<h2 style='background:purple; border:0; color:white'><center>Augmentation - MotionBlur<center><h2>

In [ ]:
transform = A.MotionBlur(
    p=1.0, blur_limit=(3, 50),
)

plot_multiple_augmentation(
    "../input/gan-getting-started/monet_jpg/990ed28f62.jpg",
    transform,
)


<a id="121"></a>
<h2 style='background:purple; border:0; color:white'><center>Augmentation - MultiplicativeNoise<center><h2>

In [ ]:
transform = A.MultiplicativeNoise(
    p=1.0, multiplier=(0.1, 5.0), per_channel=True, elementwise=False,
)

plot_multiple_augmentation(
    "../input/gan-getting-started/monet_jpg/990ed28f62.jpg",
    transform,
)
